In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import scale
from scipy.sparse import lil_matrix, csr_matrix, vstack
import pdb
import copy
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

def print_results(predictions, labels, name):
    print("{}: {}".format(name, (predictions == labels).sum() / float(len(labels))))

def pad_zeros(input_vector, output_size):
    pad = np.zeros(output_size - input_vector.shape[0])
    return np.append(input_vector, pad)

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def graph_to_degree_hist(G):
    degree_hist = np.array(nx.degree_histogram(G))
    degree_hist_padded = pad_zeros(degree_hist, nx.number_of_nodes(G))
    return degree_hist_padded

def plot_degree_histograms(data, labels, prefix=""):
    classes = sorted(list(set(labels)))
    num_classes = len(classes)
    labels = np.array(labels)
    
    f, axarr = plt.subplots(num_classes, sharex=True, sharey=True)
    f.suptitle("{}: Average Node Degree Histogram".format(prefix))
    
    for counter, class_category in enumerate(classes):
        idx = np.where(labels == class_category)
        X = data[idx]
        mean_hist = np.mean(np.array(X), axis=0)
        axarr[counter].bar(np.arange(data.shape[1]), mean_hist, label="p: "+class_category)
        
    f.subplots_adjust(hspace=0)
    for ax in axarr:
        ax.label_outer()
        ax.legend()
        
def visualization(data, labels, method="tSNE"):
    if method == "tSNE":
        viz = TSNE(n_components=2, random_state=0)
    elif method == "PCA":
        viz = PCA(n_components=2)
    data_2d = viz.fit_transform(data)
    label_classes = np.unique(labels).tolist()
    
    plt.figure(figsize=(6, 5))
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k', 'w', 'orange', 'purple']
    for i, c in zip(label_classes, colors):
        plt.scatter(data_2d[labels == i, 0], data_2d[labels == i, 1], c=c, label=i)
    plt.title("{} visualization of graph classes".format(method))
    plt.legend()
    
def load_data(train_path, test_path):
    train_list = [f for f in os.listdir(train_path) if not f.startswith('.')]
    test_list = [f for f in os.listdir(test_path) if not f.startswith('.')]
    train_list.sort()
    test_list.sort()

    graphs_train = []
    graphs_test = []
    labels_train = []
    labels_test = []

    for filename in train_list:
        graphs_train.append(nx.read_adjlist(train_path+filename))
        labels_train.append(filename.split("_")[2][:3])

    for filename in test_list:
        graphs_test.append(nx.read_adjlist(test_path+filename))
        labels_test.append(filename.split("_")[2][:3])

    return graphs_train, graphs_test, labels_train, labels_test

def WL_compute(ad_list, node_label, h):
    """
    INPUTS
    - ad_list: the graphs are stored in lists; each element is a list itself containing the adjancency lists
    - node_label: list of lists, where each entry is a list of node labels for the graph
    - h: iterations of WL

    OUTPUTS
    - K: List of kernel matrices at each iteration
    - phi_list: list of features matrices

    """
    # Total number of graphs in the dataset
    n = len(ad_list)
    
    # Total number of nodes in dataset: initialized as zero
    tot_nodes = 0


    # list of kernel matrices
    K = [0]*(h+1)
    # list of feature mtrices
    phi_list = [0] * (h+1)

    #total number of nodes in the dataset
    for i in range(n):
        tot_nodes = tot_nodes + int(len(ad_list[i]))

    
    #each column of phi will be the explicit feature representation for the graph j
    phi = lil_matrix((tot_nodes, n), dtype = np.uint32)

    # labels will be used to store the new labels
    labels = [0] * n

    #label lookup is a dictionary which will contain the mapping
    # from multiset labels (strings) to short labels (integers)
    label_lookup = {}

    # counter to create possibly new labels in the update step
    label_counter = 0

    # Note: here we are just renaming the node labels from 0,..,num_labels
    # for each graph
    for i in range(n):

        # copy the original labels
        l_aux = np.copy(node_label[i])

        # will be used to store the new labels
        labels[i] = np.zeros(len(l_aux), dtype = np.int32)

        # for each label in graph
        for j in range(len(l_aux)):
            l_aux_str = str(l_aux[j])

            # If the string do not already exist
            # then create a new short label
            if l_aux_str not in label_lookup:
                label_lookup[l_aux_str] = label_counter
                labels[i][j] = label_counter                                                              
                label_counter += 1
            else:
                labels[i][j] = label_lookup[l_aux_str]

            # node histograph of the new labels
            phi[labels[i][j],i] += 1

    L = label_counter
    print('Number of original labels %d' %L)

    #####################
    # --- Main code --- #
    #####################

    # Now we are starting with the first iteration of WL

    # features obtained from the original node (renamed) labels
    phi_list[0] = phi

    # Kernel matrix based on original features
    K[0] = phi.transpose().dot(phi).toarray().astype(np.float32)
    
    print("K original is computed")
    
    # Initialize iterations to 0
    it = 0

    # copy of the original labels: will stored the new labels
    new_labels = np.copy(labels)
    
    # until the number of iterations is less than h
    while it < h:

        # Initialize dictionary and counter 
        # (same meaning as before)        
        label_lookup = {}
        label_counter = 0

        # Initialize phi as a sparse matrix
        phi = lil_matrix((tot_nodes, n), dtype = np.int32)
        # convert it to array
        phi = phi.toarray()

        print("Iteration %d: phi is computed" % it)

        # for each graph in the dataset
        for i in range(n):

            # will store the multilabel string
            l_aux_long = np.copy(labels[i])

            # for each node in graph
            for v in range(len(ad_list[i])):

                # the new labels convert to tuple
                new_node_label = tuple([l_aux_long[v]]) 

                # form a multiset label of the node neighbors 
                new_ad = np.zeros(len(ad_list[i][v]))
                for j in range(len(ad_list[i][v])):
                    new_ad[j] = ad_list[i][v][j]
                new_ad = new_ad.astype(int)
                ad_aux = tuple([l_aux_long[j] for j in new_ad])

                # long labels: original node plus sorted neughbors
                long_label = tuple(tuple(new_node_label)+tuple(sorted(ad_aux)))
            
                # if the multiset label has not yet occurred , add
                # it to the lookup table and assign a number to it
                if long_label not in label_lookup:
                    label_lookup[long_label] = str(label_counter)
                    new_labels[i][v] = str(label_counter)
                    label_counter += 1

                # else assign it the already existing number
                else:
                    new_labels[i][v] = label_lookup[long_label]

            # count the node label frequencies
            aux = np.bincount(new_labels[i]) 
            phi[new_labels[i],i] += aux[new_labels[i]]
        
        L = label_counter
        print('Number of compressed labels %d' %L)

        # create phi for iteration it+1
        phi_sparse = lil_matrix(phi)
        phi_list[it+1] = phi_sparse

        print("Itaration %d: phi sparse saved" % it)

        # create K at iteration it+1
        K[it+1] = K[it] + phi_sparse.transpose().dot(phi_sparse).toarray().astype(np.float32)
       
        print("Iteration %d: K is computed" % it)

        # Initialize labels for the next iteration as the new just computed
        labels = copy.deepcopy(new_labels)

        # increment the iteration
        it = it + 1 

    return K, phi_list

In [2]:
# load data
dataset_path = "test_1/"
train_path = "./data/"+dataset_path+"train/"
test_path = "./data/"+dataset_path+"test/"
graphs_train, graphs_test, labels_train, labels_test = load_data(train_path, test_path)

In [3]:
ad_list_train = []
node_label_train = []
ad_list_test = []
node_label_test = []

for graph in graphs_train:
    al = graph.adjacency_list()
    nl = nx.degree(graph).values()
    nl = [str(i) for i in nl]
    ad_list_train.append(al)
    node_label_train.append(nl)

for graph in graphs_test:
    al = graph.adjacency_list()
    nl = nx.degree(graph).values()
    nl = [str(i) for i in nl]
    ad_list_test.append(al)
    node_label_test.append(nl)
    
ad_list = ad_list_train + ad_list_test
node_label_list = node_label_train + node_label_test
    
kernel_list, phi_list = WL_compute(ad_list, node_label_list, 5)

kernel_mat = np.asarray(kernel_list[-1])
train_kernel_mat = kernel_mat[:len(graphs_train), :len(graphs_train)]
test_kernel_mat = kernel_mat[len(graphs_train):, :len(graphs_train)]

print(np.asarray(kernel_mat).shape)
print(np.asarray(train_kernel_mat).shape)
print(np.asarray(test_kernel_mat).shape)

train_labels = np.array(labels_train)
test_labels = np.array(labels_test)

Number of original labels 12
K original is computed
Iteration 0: phi is computed
Number of compressed labels 3761
Itaration 0: phi sparse saved
Iteration 0: K is computed
Iteration 1: phi is computed
Number of compressed labels 19890
Itaration 1: phi sparse saved
Iteration 1: K is computed
Iteration 2: phi is computed
Number of compressed labels 20000
Itaration 2: phi sparse saved
Iteration 2: K is computed
Iteration 3: phi is computed
Number of compressed labels 20000
Itaration 3: phi sparse saved
Iteration 3: K is computed
Iteration 4: phi is computed
Number of compressed labels 20000
Itaration 4: phi sparse saved
Iteration 4: K is computed
(400, 400)
(200, 200)
(200, 200)


In [4]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

classifiers = {
    "svm": SVC()
}

parameters_search = {
    "svm": {'kernel':['precomputed'], 'C':[1]}
}

In [5]:
# select model
model = classifiers["svm"]
parameters = parameters_search["svm"]
clf = GridSearchCV(model, parameters)

# Training session 1
clf.fit(train_kernel_mat, train_labels)
train_predictions = clf.predict(train_kernel_mat).reshape(-1,1)
test_predictions = clf.predict(test_kernel_mat).reshape(-1,1)
print_results(np.ravel(train_predictions), train_labels, "Train Accuracy")
print_results(np.ravel(test_predictions), test_labels, "Test Accuracy")
print("best score: {} | best params: {}".format(clf.best_score_, clf.best_params_))

Train Accuracy: 1.0
Test Accuracy: 1.0
best score: 1.0 | best params: {'C': 1, 'kernel': 'precomputed'}
